In [1]:
import pandas as pd
import numpy as np
import nibabel as ni
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
# import seaborn as sns

In [2]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

In [3]:
df = pd.read_csv("mgh_vars.csv")

In [45]:
var_names = ["lnm_tot", "lnm_pos", "lnm_neg", "vlsm_tot", "wm_total", "lnm_tot_avg", "lnm_pos_avg", "lnm_neg_avg", "vlsm_tot_avg", "vlsm_pos_avg", "wm_avg", "sp_corr_lnm", "sp_corr_vlsm", "sp_corr_wm", "l_size"] #sp_corr_vlsm and sp_corr_lnm for sp corr vars amd "wm_total", "wm_avg", "sp_corr_wm" for wm vars
#var_names = ["lnm_tot", "lnm_pos", "lnm_neg", "vlsm_tot", "lnm_tot_avg", "lnm_pos_avg", "lnm_neg_avg", "vlsm_tot_avg", "vlsm_pos_avg", "sp_corr_lnm", "sp_corr_vlsm", "l_size"] #sp_corr_vlsm and sp_corr_lnm for sp corr vars amd "wm_total", "wm_avg", "sp_corr_wm" for wm vars

In [5]:
plt.rcParams['figure.figsize'] = [8, 6]
plt.rcParams['figure.dpi'] = 100

In [16]:
for i in range(len(var_names)):
    model = smf.ols(formula= str("nih"+"~"+var_names[i]), data=df).fit()
#     print(var_names[i])
#     print()
#     sm.graphics.plot_regress_exog(model, var_names[i])
#     plt.show()
#     print(model.summary())
#     print()

In [4]:
for i in range(len(var_names)):
    for ii in range(len(var_names)):
        if i != ii and i < ii and (i == 4 or ii == 4 or i == 10 or ii == 10):
            model = smf.ols(formula= str("nih~"+var_names[i]+"+"+var_names[ii]), data=df).fit()
#             print(var_names[i])
#             print(var_names[ii])
#             print()
#             print(model.summary())
#             print()

NameError: name 'df' is not defined

In [47]:
model = smf.ols(formula="nih~lnm_pos+vlsm_tot+wm_total", data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    nih   R-squared:                       0.278
Model:                            OLS   Adj. R-squared:                  0.274
Method:                 Least Squares   F-statistic:                     62.12
Date:                Fri, 30 Apr 2021   Prob (F-statistic):           5.43e-34
Time:                        21:52:17   Log-Likelihood:                -1525.6
No. Observations:                 488   AIC:                             3059.
Df Residuals:                     484   BIC:                             3076.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.2979      0.298     11.063      0.0

In [35]:
all_vars = "+".join(var_names)
model = smf.ols(formula= str("nih~"+all_vars), data=df).fit()
# print(model.summary())
# print()

### Level 1 - Simple

In [4]:
simple = "nih~l_size+vlsm_tot_avg+lnm_pos_avg+lnm_neg_avg+sp_corr_lnm+wm_avg"
model = smf.ols(formula=simple, data=df).fit()
# print(model.summary())

In [5]:
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100

In [6]:
sim_ls = ["l_size","vlsm_tot_avg","lnm_pos_avg","lnm_neg_avg","sp_corr_lnm", "wm_avg"]
# for i in range(len(sim_ls)):
#     sm.graphics.plot_regress_exog(model, sim_ls[i])
#     plt.show()

### Level 2 - Lesion Size * Vars

In [7]:
df["l_size_x_vlsm_tot_avg"] = df["l_size"] * df["vlsm_tot_avg"]
df["l_size_x_lnm_pos_avg"] = df["l_size"] * df["lnm_pos_avg"]
df["l_size_x_lnm_neg_avg"] = df["l_size"] * df["lnm_neg_avg"]
df["l_size_x_sp_corr_lnm"] = df["l_size"] * df["sp_corr_lnm"]
df["l_size_x_wm_avg"] = df["l_size"] * df["wm_avg"]

In [8]:
size_multi = simple+"+l_size_x_vlsm_tot_avg+l_size_x_lnm_pos_avg+l_size_x_lnm_neg_avg+l_size_x_sp_corr_lnm+l_size_x_wm_avg"
model = smf.ols(formula=size_multi, data=df).fit()
# print(model.summary())

In [10]:
sm_ls = sim_ls+["l_size_x_vlsm_tot_avg","l_size_x_lnm_pos_avg","l_size_x_lnm_neg_avg","l_size_x_sp_corr_lnm","l_size_x_wm_avg"]
# for i in range(len(sm_ls)):
#     sm.graphics.plot_regress_exog(model, sm_ls[i])
#     plt.show()

### Level 3 - Totals

In [11]:
totals = simple+"+vlsm_tot+lnm_pos+lnm_neg+wm_total+l_size_x_sp_corr_lnm"
model = smf.ols(formula=totals, data=df).fit()
# print(model.summary())

In [13]:
tt_ls = sim_ls+["vlsm_tot","lnm_pos","lnm_neg","wm_total","l_size_x_sp_corr_lnm"]
# for i in range(len(tt_ls)):
#     sm.graphics.plot_regress_exog(model, tt_ls[i])
#     plt.show()

## Training and Testing

In [10]:
tr = pd.read_csv("acute_vars.csv")
te = pd.read_csv("mgh_vars.csv")

In [12]:
for1 = "nih~l_size+vlsm_tot_avg+lnm_pos_avg+lnm_neg_avg"
model = smf.ols(formula=for1, data=tr).fit()
# print(model.summary())

In [101]:
for1 = "nih~l_size+vlsm_tot_avg+lnm_pos_avg+lnm_neg_avg"
model = smf.ols(formula=for1, data=df).fit()
# print(model.summary())

In [97]:
yhat = np.array(model.predict(te[["l_size", "vlsm_tot_avg", "lnm_pos_avg", "lnm_neg_avg"]]))
y = np.array(te["nih"])

In [13]:
def coeff_det(y, yhat):
    ybar = np.mean(y)
    sse = np.sum(np.square(np.subtract(y, yhat)))
    sst = np.sum(np.square(y - ybar))
    return (1 - sse/sst)

In [102]:
r2 = coeff_det(y, yhat)
# print(r2)

In [62]:
# for2 = "nih~l_size+vlsm_tot_avg+sp_corr_lnm+sp_corr_vlsm"
# model = smf.ols(formula=for2, data=tr).fit()
# print(model.summary())

In [15]:
yhat = np.array(model.predict(te[["l_size", "vlsm_tot_avg", "sp_corr_lnm", "sp_corr_vlsm"]]))
y = np.array(te["nih"])

In [16]:
r2 = coeff_det(y, yhat)
print(r2)

0.2485004698527623


In [18]:
for2 = "nih~l_size+vlsm_tot_avg+sp_corr_lnm+sp_corr_vlsm"
model = smf.ols(formula=for2, data=te).fit()
# print(model.summary())

In [63]:
# sc_only = "nih~sp_corr_lnm+sp_corr_vlsm"
# model = smf.ols(formula=sc_only, data=tr).fit()
# print(model.summary())

In [20]:
yhat = np.array(model.predict(te[["sp_corr_lnm", "sp_corr_vlsm"]]))
y = np.array(te["nih"])

In [21]:
r2 = coeff_det(y, yhat)
print(r2)

0.03754877949130486


In [64]:
# sc_only = "nih~sp_corr_lnm+sp_corr_vlsm"
# model = smf.ols(formula=sc_only, data=te).fit()
# print(model.summary())

In [14]:
avg_only = "nih~vlsm_tot_avg+lnm_pos_avg+lnm_neg_avg"
model = smf.ols(formula=avg_only, data=tr).fit()
# print(model.summary())

In [15]:
yhat = np.array(model.predict(te[["vlsm_tot_avg", "lnm_pos_avg", "lnm_neg_avg"]]))
y = np.array(te["nih"])

In [16]:
r2 = coeff_det(y, yhat)
print(r2)

0.08067500384147286


In [18]:
avg_only = "nih~vlsm_tot_avg+lnm_pos_avg+lnm_neg_avg"
model = smf.ols(formula=avg_only, data=te).fit()
# print(model.summary())

In [65]:
# sc_size = "nih~sp_corr_lnm+sp_corr_vlsm+l_size"
# model = smf.ols(formula=sc_size, data=tr).fit()
# print(model.summary())

In [24]:
yhat = np.array(model.predict(te[["sp_corr_lnm", "sp_corr_vlsm", "l_size"]]))
y = np.array(te["nih"])

In [25]:
r2 = coeff_det(y, yhat)
print(r2)

0.24544329616263638


In [66]:
# sc_size = "nih~sp_corr_lnm+sp_corr_vlsm+l_size"
# model = smf.ols(formula=sc_size, data=te).fit()
# print(model.summary())

### Memory Specific 

In [22]:
mem_df = pd.read_csv("year_mem.csv")

In [14]:
mem_NDS = ["mem_NDS_tot", "mem_NDS_pos_only", "mem_NDS_neg_only", "mem_NDS_tot_avg", "mem_NDS_pos_avg", "mem_NDS_neg_avg", "mem_NDS_true_tot", "mem_NDS_true_pos_only", "mem_NDS_true_neg_only", "mem_NDS_true_tot_avg", "mem_NDS_true_pos_avg", "mem_NDS_true_neg_avg", "mem_sp_corr"]

In [24]:
for x in mem_NDS:
    model = smf.ols(formula= str("revised_memv_factor"+"~"+x), data=mem_df).fit()
#     print(x)
#     print()
#     print(model.summary())
#     print()

In [26]:
for x in mem_NDS:
    model = smf.ols(formula= str("revised_mems_factor"+"~"+x), data=mem_df).fit()
#     print(x)
#     print()
#     print(model.summary())
#     print()

In [27]:
all_vars = pd.read_csv("year_full_beh.csv")
subjects = list(mem_df["study_id"])
nih = []
for s in subjects:
    nih.append(float(all_vars.loc[all_vars.study_id == s, "nih_total"]))
mem_df["nih"] = nih

In [29]:
for x in mem_NDS:
    model = smf.ols(formula= str("nih"+"~"+x), data=mem_df).fit()
#     print(x)
#     print()
#     print(model.summary())
#     print()

### Corrected Spatial Correlations

In [9]:
mgh = pd.read_csv("mgh_vars.csv")
model = smf.ols(formula="nih~sp_corr_lnm", data=mgh).fit()
# print(model.summary())
model = smf.ols(formula="nih~sp_corr_vlsm", data=mgh).fit()
# print(model.summary())

### CV Correlations and Shrinkage

In [82]:
dataset = "mgh"

In [83]:
corb = pd.read_csv(dataset+"_vars.csv")
mgh = pd.read_csv(dataset+"_vars_cv.csv")

In [84]:
r2vals = pd.DataFrame(columns=["corb1", "mgh1", "corb2", "mgh2", "corb_all", "mgh_all"])

In [85]:
corb1_r = []
mgh1_r = []
for i in range(len(var_names)):
    model_corb = smf.ols(formula= str("nih"+"~"+var_names[i]), data=corb).fit()
    model_mgh = smf.ols(formula= str("nih"+"~"+var_names[i]), data=mgh).fit()
    corb1_r.append(model_corb.rsquared)
    mgh1_r.append(model_mgh.rsquared)

In [86]:
corb2_r = []
mgh2_r = []
for i in range(len(var_names)):
    for ii in range(len(var_names)):
        if i != ii and i < ii:
            model_corb = smf.ols(formula= str("nih~"+var_names[i]+"+"+var_names[ii]), data=corb).fit()
            model_mgh = smf.ols(formula= str("nih~"+var_names[i]+"+"+var_names[ii]), data=mgh).fit()
            corb2_r.append(model_corb.rsquared)
            mgh2_r.append(model_mgh.rsquared)

In [87]:
corb_all = corb1_r + corb2_r
mgh_all = mgh1_r + mgh2_r

In [88]:
r2vals["corb_all"] = pd.Series(corb_all)
r2vals["mgh_all"] = pd.Series(mgh_all)
r2vals["corb2"] = pd.Series(corb2_r)
r2vals["mgh2"] = pd.Series(mgh2_r)
r2vals["corb1"] = pd.Series(corb1_r)
r2vals["mgh1"] = pd.Series(mgh1_r)

In [52]:
# print("Single Regression")
# model = smf.ols(formula="mgh1~corb1", data=r2vals).fit()
# print(model.summary())
# print()
# print("Double Regression")
# model = smf.ols(formula="mgh2~corb2", data=r2vals).fit()
# print(model.summary())
# print()
# print("All Regression")
# model = smf.ols(formula="mgh_all~corb_all", data=r2vals).fit()
# print(model.summary())
# print()

In [89]:
corb_vars = ["corb_"+k for k in var_names]
mgh_vars = ["mgh_"+k for k in var_names]
all_vars = corb_vars + mgh_vars
vars_df = pd.DataFrame(columns=all_vars)

In [91]:
# for k in var_names:
#     vars_df["corb_"+k] = corb[k]
#     vars_df["mgh_"+k] = mgh[k]
#     model = smf.ols(formula=str("mgh_"+k+"~corb_"+k), data=vars_df).fit()
#     print(k)
#     print(model.summary())
#     print()

### Find Highest R-squared

In [33]:
dataset = "mgh"

In [34]:
corb = pd.read_csv(dataset+"_vars.csv")
mgh = pd.read_csv(dataset+"_vars_cv.csv")

In [35]:
corb1_r = []
mgh1_r = []
for i in range(len(var_names)):
    model_corb = smf.ols(formula= str("nih"+"~"+var_names[i]), data=corb).fit()
    model_mgh = smf.ols(formula= str("nih"+"~"+var_names[i]), data=mgh).fit()
    corb1_r.append((model_corb.rsquared, var_names[i]))
    mgh1_r.append((model_mgh.rsquared, var_names[i]))
corb1_r = sorted(corb1_r, reverse=True)
mgh1_r = sorted(mgh1_r, reverse=True)

In [36]:
print(corb1_r[0])
print()
print(mgh1_r[0])

(0.2958384043901142, 'sp_corr_wm')

(0.3620016454635848, 'sp_corr_wm')


In [37]:
corb2_r = []
mgh2_r = []
for i in range(len(var_names)):
    for ii in range(len(var_names)):
        if i != ii and i < ii:
            model_corb = smf.ols(formula= str("nih~"+var_names[i]+"+"+var_names[ii]), data=corb).fit()
            model_mgh = smf.ols(formula= str("nih~"+var_names[i]+"+"+var_names[ii]), data=mgh).fit()
            corb2_r.append((model_corb.rsquared, var_names[i], var_names[ii]))
            mgh2_r.append((model_mgh.rsquared, var_names[i], var_names[ii]))
corb2_r = sorted(corb2_r, reverse=True)
mgh2_r = sorted(mgh2_r, reverse=True)

In [38]:
print(corb2_r[0:5])
print()
print(mgh2_r[0:5])

[(0.3226258874752794, 'sp_corr_wm', 'l_size'), (0.3165408461102205, 'lnm_neg', 'sp_corr_wm'), (0.314107017670779, 'lnm_pos', 'sp_corr_wm'), (0.3137732438987513, 'wm_total', 'sp_corr_wm'), (0.31093574459464435, 'wm_avg', 'sp_corr_wm')]

[(0.37495585589876046, 'lnm_neg', 'sp_corr_wm'), (0.36997505396774477, 'sp_corr_wm', 'l_size'), (0.3661672314809553, 'wm_total', 'sp_corr_wm'), (0.3660145346508962, 'wm_avg', 'sp_corr_wm'), (0.36590288990391195, 'sp_corr_lnm', 'sp_corr_wm')]


### Seed Region

In [38]:
sr_df = pd.read_csv("mgh_seed_region_212.csv")
nih = pd.read_csv("mgh_vars.csv")["nih"]
subjects = pd.read_csv("mgh_vars.csv")["subject"]
sr_df["nih"] = nih

In [39]:
sr_vars = ["sr_total", "sr_pos", "sr_neg", "sr_total_avg", "sr_pos_avg", "sr_neg_avg", "sr_sp_corr"]

In [32]:
for i in range(len(sr_vars)):
    model = smf.ols(formula=str("nih~"+sr_vars[i]), data=sr_df).fit()
#     print(sr_vars[i])
#     print(model.summary())
#     print()

In [52]:
combined_cols = ["subject", "nih", "vlsm_total", "lnm_total", "lnm_pos", "lnm_neg", "total_212", "pos_212", "neg_212", "total_m30", "pos_m30", "neg_m30"]
combined_df = pd.DataFrame(columns=combined_cols)

In [53]:
combined_df["subject"] = subjects
combined_df["nih"] = nih

In [54]:
sr_212 = pd.read_csv("mgh_seed_region_212.csv")
sr_minus30 = pd.read_csv("mgh_seed_region_-30.csv")
mgh_df = pd.read_csv("mgh_vars.csv")

In [55]:
combined_df["vlsm_total"] = mgh_df["vlsm_tot"]
combined_df["lnm_total"] = mgh_df["lnm_tot"]
combined_df["lnm_pos"] = mgh_df["lnm_pos"]
combined_df["lnm_neg"] = mgh_df["lnm_neg"]

In [56]:
combined_df["total_212"] = sr_212["sr_total"]
combined_df["pos_212"] = sr_212["sr_pos"]
combined_df["neg_212"] = sr_212["sr_neg"]

In [57]:
combined_df["total_m30"] = sr_minus30["sr_total"]
combined_df["pos_m30"] = sr_minus30["sr_pos"]
combined_df["neg_m30"] = sr_minus30["sr_neg"]

In [69]:
pred = combined_cols[2:]
for i in range(len(pred)):
    for ii in range(len(pred)):
        if i > ii:
            model = smf.ols(formula=str("nih~"+pred[i]+"+"+pred[ii]), data=combined_df).fit()
#             print(pred[i])
#             print(pred[ii])
#             print(model.summary())
#             print()

In [67]:
pred = combined_cols[2:]
for i in range(len(pred)):
    for ii in range(len(pred)):
        for iii in range(len(pred)):
            if i > ii and ii > iii:
                model = smf.ols(formula=str("nih~"+pred[i]+"+"+pred[ii]+"+"+pred[iii]), data=combined_df).fit()
#                 print(pred[i])
#                 print(pred[ii])
#                 print(pred[iii])
#                 print(model.summary())
#                 print()

In [13]:
model = smf.ols(formula="nih~lnm_pos+vlsm_tot+l_size", data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                    nih   R-squared:                       0.277
Model:                            OLS   Adj. R-squared:                  0.273
Method:                 Least Squares   F-statistic:                     61.84
Date:                Wed, 05 May 2021   Prob (F-statistic):           7.36e-34
Time:                        18:51:45   Log-Likelihood:                -1525.9
No. Observations:                 488   AIC:                             3060.
Df Residuals:                     484   BIC:                             3077.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.2451      0.303     10.709      0.0